In [ ]:
import re

In [ ]:
gifu = pd.read_csv('input/stations_gifu.csv', index_col=0)
aichi = pd.read_csv('input/stations_aichi.csv', index_col=0)

In [ ]:
gifu.index = 'g_' + gifu.index.astype(str)
aichi.index = 'a_' + aichi.index.astype(str)

In [ ]:
gifu['路線ID'] = gifu['路線ID'].astype(str)
aichi['路線ID'] = aichi['路線ID'].astype(str)

In [ ]:
gifu['路線ID'] = 'g_' + gifu['路線ID']
aichi['路線ID'] = 'a_' + aichi['路線ID']

In [ ]:
stations = pd.concat([gifu, aichi], axis=0)

In [ ]:
stations

In [ ]:
#.isalnum()は全角文字に対応してない
alnumReg = re.compile(r'^[a-zA-Z0-9]+$')
def isalnum(s):
    return alnumReg.match(s) is not None

rm_row = []
for ind in range(stations.shape[0]):
    for col in range(stations.shape[1]-1):
#        print(ind, col)
        stations.iloc[ind, col] = stations.iloc[ind, col].replace(',', '')
        while not isalnum(stations.iloc[ind, col]):
            if stations.iloc[ind, col] == '':
                print(ind, col)
                rm_row.append(stations.iloc[ind].name)
                break
            else:
                stations.iloc[ind, col] = stations.iloc[ind, col][:-1]

In [ ]:
stations.iloc[106]

In [ ]:
rm_row

In [ ]:
stations = stations.drop(rm_row, axis=0)

In [ ]:
stations

In [ ]:
rosen = stations['路線ID'] 
stations = stations.drop(['路線ID'], axis=1)
stations = stations.astype(float)
stations = pd.concat([stations, rosen], axis=1)

In [ ]:
stations = stations.drop(['男性人口比率', '1人世帯比率', '生徒学生数', '0～9歳人口(比率)', '10～19歳人口(比率)', '20～29歳人口(比率)', '30～39歳人口(比率)', '40～49歳人口(比率)', '50～59歳人口(比率)', '60～69歳人口(比率)', '70～79歳人口(比率)', '80歳以上人口(比率)'], axis=1)

In [ ]:
stations

In [ ]:
stations.to_csv('input/stations_gifu-aichi_edited.csv')

In [ ]:
stations = pd.read_csv('input/stations_gifu-aichi_edited.csv', index_col=0)

In [ ]:
X = stations.drop(['飲食店事業所数'], axis=1)
y = stations['飲食店事業所数']

In [ ]:
X = pd.get_dummies(X)

In [ ]:
X

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# scikit-learn

In [ ]:
import lightgbm as lgb
model = lgb.LGBMClassifier()
model.fit(X_train, y_train)

In [ ]:
model.score(X_train, y_train)

In [ ]:
model.feature_importances_

In [ ]:
X.columns

In [ ]:
predict = model.predict(X_test)

In [ ]:
from sklearn.metrics import r2_score
r2_score(predict, y_test)

In [ ]:
fig, ax = plt.subplots()
ax.scatter(y_test, predict, edgecolors=(0, 0, 0))
ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=1)
ax.set_xlabel('Measured')
ax.set_ylabel('Predicted')
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.scatter(predict, y_test, edgecolors=(0, 0, 0))
ax.plot([predict.min(), predict.max()], [predict.min(), predict.max()], 'k--', lw=1)
ax.set_xlabel('Predicted')
ax.set_ylabel('Measured')
plt.show()

In [ ]:
from sklearn.linear_model import LogisticRegression
log = LogisticRegression()
log.fit(X_train, y_train)

In [ ]:
log.score(X_train, y_train)

In [ ]:
log.coef_[0]

In [ ]:
log.intercept_[0]

In [ ]:
predict = log.predict(X_test)

In [ ]:
r2_score(predict, y_test)

In [ ]:
fig, ax = plt.subplots()
ax.scatter(y_test, predict, edgecolors=(0, 0, 0))
ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=1)
ax.set_xlabel('Measured')
ax.set_ylabel('Predicted')
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.scatter(predict, y_test, edgecolors=(0, 0, 0))
ax.plot([predict.min(), predict.max()], [predict.min(), predict.max()], 'k--', lw=1)
ax.set_xlabel('Predicted')
ax.set_ylabel('Measured')
plt.show()

# pystan

In [ ]:
import pystan

In [ ]:
stations = pd.read_csv('input/stations_gifu-aichi_edited.csv', index_col=0)

In [ ]:
#X = stations.drop(['飲食店事業所数'], axis=1)
X = stations.drop(['飲食店事業所数', '路線ID'], axis=1)
y = stations['飲食店事業所数']

In [ ]:
X.columns

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
N_train = X_train.shape[0]
M = X_train.shape[1]
Y_train = list(y_train.values)
N_test = X_test.shape[0]
X_test = list(X_test.values)

data = dict(
    N_train=N_train,
    M=M,
    X_train=X_train,
    Y_train=Y_train,
    N_test=N_test,
    X_test=X_test,
)

In [ ]:
model = '''
data {
    int N_train;
    int M;
    matrix<lower=0>[N_train, M] X_train;
    vector<lower=0>[N_train] Y_train;
    int N_test;
    matrix<lower=0>[N_test, M] X_test;
}

parameters {
    real b0;
    vector[M] b;
    real<lower=0> s;
    real<lower=0> s_b;
}

model {
    b0 ~ double_exponential(0, s_b);
    b ~ double_exponential(0, s_b);
    s ~ normal(0, 5);
    Y_train ~ normal(b0 + X_train*b, s);
}

generated quantities{
#    vector<lower=0>[N_test] predict;
    vector[N_test] predict;
    for (i in 1:N_test)
        predict[i] = normal_rng(b0 + X_test[i]*b, s);
}
'''

In [ ]:
fit = pystan.stan(model_code=model, data=data, chains=3, iter=10000, warmup=1000, thin=1)

In [ ]:
fit

In [ ]:
fit.extract()['predict'][0]

In [ ]:
y_test

In [ ]:
predict = fit.extract()['predict'][0]

In [ ]:
fig, ax = plt.subplots()
ax.scatter(predict, y_test, edgecolors=(0, 0, 0))
ax.plot([predict.min(), predict.max()], [predict.min(), predict.max()], 'k--', lw=1)
ax.set_xlabel('Predicted')
ax.set_ylabel('Measured')
plt.show()